In [1]:
%load_ext autoreload
%autoreload 2

In [73]:
import numpy as np
import cv2

# Set-up

These values of intrinsics were found out by regressing `skeletons_image.txt` and `skeletons_world.txt` to find out the transformation matrix between $(u,v)$ and $(X_w,Y_w,Z_w)$.

In [3]:
fx, fy, cx, cy = (440.44232, -461.0357, -0.00015258789, 3.0517578e-05)

`g` - Gesture #

`f` - Finger #

`s` - Subject #

`e` - Essai #

In [29]:
g = 1
f = 1
s = 2
e = 1

ESSAI = f'./dataset/gesture_{g}/finger_{f}/subject_{s}/essai_{e}/'

In [30]:
true_uvd = np.loadtxt(f'{ESSAI}/skeletons_image.txt', dtype=np.float16)
true_xyz = np.loadtxt(f'{ESSAI}/skeletons_world.txt', dtype=np.float16)

true_uvd = true_uvd.reshape((-1, 22, 2))
true_uvd = np.delete(true_uvd, 1, 1) # 3, 4

true_xyz = true_xyz.reshape((-1, 22, 3))
true_xyz = np.delete(true_xyz, 1, 1) # 4, 5, 6

true_uvd.shape, true_xyz.shape

((77, 21, 2), (77, 21, 3))

# Inference

In [31]:
from srn import srn
handpose = srn.handpose

In [34]:
pred_xyz, pred_uvd = handpose.run(ESSAI)

## Preprocess

Flip image coordinates
$$ u = 640 - u' $$

## Calculate world coordinates

$$ world = \frac{image}{f} Z_{world} $$

In [36]:
pred_U = pred_uvd[:,:,0].flatten()
pred_V = pred_uvd[:,:,1].flatten()
pred_D = pred_uvd[:,:,2].flatten()

In [48]:
calc_X = pred_U / fx * pred_D * 0.00092
calc_Y = pred_V / fy * pred_D * 0.00092
calc_Z = pred_D * 0.00092

In [38]:
true_X = true_xyz[:,:,0].flatten()
true_Y = true_xyz[:,:,1].flatten()
true_Z = true_xyz[:,:,2].flatten()
true_U = true_uvd[:,:,0].flatten()
true_V = true_uvd[:,:,1].flatten()
true_D = true_Z

## Examine

### Scaling of true depth and predicted depth
What might be the scaling? 1111 had 0.00092

In [58]:
(true_D / pred_D)

array([0.00088013, 0.00089107, 0.00093826, ..., 0.00096218, 0.00094196,
       0.00096934], dtype=float32)

In [59]:
(true_D / pred_D).mean()

0.0009766514

## Errors

Error in $X_w$

In [49]:
np.mean(np.abs(true_X - calc_X))

0.02743401

Error in $Y_w$

In [50]:
np.mean(np.abs(true_Y - calc_Y))

0.02357011

Error in $Z_w$

In [51]:
np.mean(np.abs(true_Z - calc_Z))

0.027494956

Error in $u$

In [42]:
np.mean(np.abs(true_U - pred_U))

12.82787

Error in $v$

In [43]:
np.mean(np.abs(true_V - pred_V))

22.276278

## Plots

### Plotting pred_uvd

In [ ]:
for frameno, poses in enumerate(pred_uvd):
    img = cv2.imread(ESSAI + f'/{frameno}_depth.png', cv2.IMREAD_ANYDEPTH)
    plt.imshow(img)
    for u, v, d in poses:
        c = plt.Circle((u, v), radius=2.5, color='r')
        plt.gca().add_patch(c)
    plt.show()

### Plotting true_uvd

In [ ]:
for frameno, poses in enumerate(true_uvd):
    img = cv2.imread(ESSAI + f'/{frameno}_depth.png', cv2.IMREAD_ANYDEPTH)
    plt.imshow(img)
    for u, v in poses:
        c = plt.Circle((u, v), radius=2.5, color='r')
        plt.gca().add_patch(c)
    plt.show()